# Chapter 7 - Open-source Frameworks: Building Conversational AI with Amazon Bedrock

## Overview
This notebook demonstrates how to build conversational AI applications using open-source frameworks integrated with Amazon Bedrock. We'll explore how to leverage frameworks like LangChain and LlamaIndex for building sophisticated AI applications.

## Introduction
This notebook demonstrates how to build conversational AI applications using Amazon Bedrock's foundation models. We'll explore two distinct approaches:
1. A basic chatbot without context memory
2. A context-aware chatbot using Retrieval-Augmented Generation (RAG)


## Prerequisites
- AWS account with Amazon Bedrock access
- Access to Amazon Titan and Nova foundation models
- PDF document for knowledge extraction

## Setup

### Install Required Packages

In [ ]:
# Install required packages
%pip install --upgrade langchain langchain_aws langchain_community "faiss-cpu>=1.7,<2" pypdf boto3

In [ ]:
# Import necessary libraries
import warnings
from io import StringIO
import sys
import textwrap
import os
import boto3

In [ ]:
# Configure Bedrock client
boto3_bedrock = boto3.client('bedrock-runtime')
warnings.filterwarnings('ignore')

In [ ]:
# Helper function for text wrapping
def print_ww(*args, width: int = 100, **kwargs):
    """Like print(), but wraps output to `width` characters (default 100)"""
    buffer = StringIO()
    try:
        _stdout = sys.stdout
        sys.stdout = buffer
        print(*args, **kwargs)
        output = buffer.getvalue()
    finally:
        sys.stdout = _stdout
    for line in output.splitlines():
        print("\n".join(textwrap.wrap(line, width=width)))

## Basic Chatbot Without Context

### Setup Conversational Model

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_aws import ChatBedrock
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.chat_message_histories import ChatMessageHistory

In [ ]:
# Set up the LLM model
modelId = "us.amazon.nova-lite-v1:0"
nova_llm = ChatBedrock(
    model_id=modelId, 
    client=boto3_bedrock,
    model_kwargs={'temperature': 0.5, 'max_tokens': 700}
)

### Create Conversation Chain

In [ ]:
# Create conversation chain using the modern approach
prompt = ChatPromptTemplate.from_template("""
System: The following is a friendly conversation between a knowledgeable helpful assistant and a customer. 
The assistant is talkative and provides lots of specific details from its context.

{history}
Human: {input}
Assistant:
""")

In [ ]:
# Create the runnable chain
chain = prompt | nova_llm

# Setup for conversation history
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversation_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [ ]:
# Function to interact with the chatbot
def chat(input_text, session_id="default"):
    response = conversation_with_history.invoke(
        {"input": input_text},
        config={"configurable": {"session_id": session_id}}
    )
    return response

### Test Basic Chatbot

In [ ]:
# Test the basic chatbot
try:
    response = chat("Hi there!")
    print_ww(response)
except ValueError as error:
    if "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubleshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

In [ ]:
# Try another conversation turn
print_ww(chat("I need a plan to learn generative AI in 30 days"))

In [ ]:
# Ask for course suggestions
print_ww(chat("Great, can you help in suggesting some courses"))

In [ ]:
# End conversation
print_ww(chat("Thank you for the inputs. Will get back if I need some help"))

## Context-Aware Chatbot with RAG

### Setup Text Embeddings and Vector Store

In [ ]:
from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [ ]:
# Create embeddings using Titan embedding model
br_embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v1", 
    client=boto3_bedrock
)

### Process Document and Create Vector Store

In [ ]:
# Load the PDF file
# data
# Note: Ensure the PDF file exists in your environment or adjust the path accordingly
pdf_loader = PyPDFLoader("data/generative-ai-guide-1.pdf")
documents = pdf_loader.load()

In [ ]:
# Split the PDF into chunks using the more modern recursive splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
data = text_splitter.split_documents(documents)
print(f"Documents: after split and chunking size={len(data)}")

In [ ]:
# Create FAISS vector store from documents
vectorstore_faiss = FAISS.from_documents(
    documents=data,
    embedding=br_embeddings
)
print(f"vectorstore_faiss: created successfully")

### Build RAG Chain

In [ ]:
# Create a retriever from the vector store
retriever = vectorstore_faiss.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

In [ ]:
# Create a prompt template for RAG
rag_prompt_template = """
Answer the question based on the following context:

{context}

Question: {input}

Answer:
"""
rag_prompt = ChatPromptTemplate.from_template(rag_prompt_template)

In [ ]:
# Create the RAG chain using modern approach
document_chain = create_stuff_documents_chain(titan_llm, rag_prompt)
rag_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
# Function to query the RAG system
def ask_rag(question):
    result = rag_chain.invoke({"input": question})
    print_ww(result["answer"])
    return result

### Test RAG-Based Chatbot

In [ ]:
# Test RAG queries
print("Question 1: What are the tips provided by adobe stock that can approve the generative AI images for sale?")
ask_rag("What are the tips provided by adobe stock that can approve the generative AI images for sale")

In [ ]:
print("\nQuestion 2: Can you elaborate on how to make money?")
ask_rag("Can you elaborate on how do you make money")

In [ ]:
print("\nQuestion 3: What happens if the image has a person?")
ask_rag("What happens if the image has a person")

# Conclusion

In this notebook, we explored two powerful approaches to building conversational AI systems using Amazon Bedrock foundation models. By implementing both a basic conversational agent and a more sophisticated RAG-based knowledge assistant, we've demonstrated the flexibility and capabilities of modern AI systems for different use cases.

The basic chatbot showcased how Amazon Nova can maintain conversational context and generate coherent, helpful responses across multiple turns of dialogue. This approach works well for general-purpose interactions where broad knowledge and conversational fluency are the primary requirements.

Our RAG-enhanced chatbot built with Amazon Titan took capabilities further by grounding responses in specific document knowledge. This approach demonstrated how to create AI assistants that can provide accurate, relevant information from particular sources - a crucial capability for enterprise applications where factual precision matters.

Key takeaways from this exercise include:

1. **Model Selection Matters**: Different models have different strengths - Nova for conversational fluency, Titan for knowledge tasks.

2. **Context Management**: Whether through conversation history or document retrieval, managing context is essential for coherent interactions.

3. **Vector Embeddings**: Semantic search using embeddings provides a powerful way to match user questions with relevant information.

4. **Prompt Engineering**: The structure and content of prompts significantly influence the quality and relevance of AI responses.

5. **Integration Power**: Combining foundation models with retrieval systems creates AI applications that are both flexible and reliable.

Amazon Bedrock provides a robust foundation for building these applications, offering both the model variety and the operational infrastructure needed for production deployments. As foundation models continue to evolve, the techniques demonstrated in this notebook will become increasingly valuable for creating AI systems that can understand, reason, and communicate effectively across a wide range of domains and use cases.